# Proyecto en Javascript

Una vez vistos todos los contenidos de la parte de Javascript, estamos listos para hacer un proyecto entero. No nos detendremos a explicar el porqué de los pasos a seguir, tan solo a hacer referencia a los capítulos anteriores. 

[Video de proyecto en JS](https://youtu.be/gFOOsefQMlg)

## Configuración inicial

En realidad, para empezar a hacer algo en Javascript en el frontend, tan solo necesitamos un archivo HTML y uno JS. Se puede empezar por ahí e ir incorporando las distintas librerias, bundlers y demás. También se puede empezar con el esqueleto del CI/CD completo y luego empezar a programar. Empezaremos de una manera "mixta", configurando `Vite` y los archivos y servicios que nos afectan a la programación desde el principio e iremos instalando y configurando conforme sea necesario. 

### Vite

Ejecutamos los comandos básicos:

```bash
npm create vite@latest my-app -- --template vanilla
cd my-app
npm install
npm run dev
```

Con esto ya tenemos una plantilla con la que comenzar. Eliminaremos el código de ejemplo, las imágenes y el css de ejemplo para empezar desde cero. Este puede ser un buen momento para instalar algun framework de estilos como `Bootstrap`. En nuesto caso, como no nos interesa tanto la parte visual, vamos a instalarlo y así simplificamos ese apartado:

https://getbootstrap.com/docs/5.2/getting-started/vite/ 


Podemos crear una estructura html mínima para el `index.html` a partir de la plantilla proporcionada por Vite:

```html
<!doctype html>
<html lang="en">
  <head>
    <meta charset="UTF-8" />
    <link rel="icon" type="image/svg+xml" href="/vite.svg" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <title>Vite App</title>
  </head>
  <body>
    <div id="app">
      <div id="menu"></div>
      <div id="container" class="container"></div>
    </div>
    <script type="module" src="/main.js"></script>
  </body>
</html>
```

Puesto que ya tenemos `Bootstrap`, crearemos el menú más simple, un `navbar` en views/views.js:

```javascript
const buildMenu = () => {
  const divWrapper = document.createElement("div");
  const menu = `<nav class="navbar navbar-expand-lg bg-light">
  <div class="container-fluid">
    <a class="navbar-brand" href="#">Movies 2024</a>
    <button class="navbar-toggler" type="button" data-bs-toggle="collapse" data-bs-target="#navbarSupportedContent" aria-controls="navbarSupportedContent" aria-expanded="false" aria-label="Toggle navigation">
      <span class="navbar-toggler-icon"></span>
    </button>
    <div class="collapse navbar-collapse" id="navbarSupportedContent">
      <ul class="navbar-nav me-auto mb-2 mb-lg-0">
        <li class="nav-item">
          <a class="nav-link active" aria-current="page" href="#">Home</a>
        </li>
        <li class="nav-item">
          <a class="nav-link" href="#">Link</a>
        </li>
        <li class="nav-item dropdown">
          <a class="nav-link dropdown-toggle" href="#" role="button" data-bs-toggle="dropdown" aria-expanded="false">
            Dropdown
          </a>
          <ul class="dropdown-menu">
            <li><a class="dropdown-item" href="#">Action</a></li>
            <li><a class="dropdown-item" href="#">Another action</a></li>
            <li><hr class="dropdown-divider"></li>
            <li><a class="dropdown-item" href="#">Something else here</a></li>
          </ul>
        </li>
        <li class="nav-item">
          <a class="nav-link disabled">Disabled</a>
        </li>
      </ul>
      <form class="d-flex" role="search">
        <input class="form-control me-2" type="search" placeholder="Search" aria-label="Search">
        <button class="btn btn-outline-success" type="submit">Search</button>
      </form>
    </div>
  </div>
</nav>`;
  divWrapper.innerHTML = menu;
  return divWrapper.querySelector("nav");
};
```

y en main.js:

```javascript
import "./styles.scss";
//import * as bootstrap from "bootstrap";
import { buildMenu } from "./views/views";

document.addEventListener("DOMContentLoaded", async () => {
  const menuDiv = document.querySelector("#menu");
  const containerDiv = document.querySelector("#container");
  containerDiv.innerHTML = "";
  menuDiv.append(buildMenu());
});
```

## Estructura del proyecto

El proyecto va a ser muy básico, así que no necesitamos una gran arquitectura. Separaremos la conexión al backend de la vista y los controladores. Por tanto, inicialmente podemos crear carpetas para `modelos` o `servicios` y para las `vistas`. Los modelos se conectarán a la base de datos y servirán los datos de forma reactiva, es decir, con RxJS. Las vistas crearán elementos del `DOM` mediante template literals y serán reactivas. Haremos un CRUD con autenticación en el que el backend será `Supabase`. 

## Backend básico con Supabase

Nos damos de alta en Supabase y creamos un proyecto nuevo. Una vez creado, creamos las tablas de la base de datos y ponemos algunos datos. En nuestro caso, he creado inicialmente una tabla `movies` con el contenido de una base de datos de unas 45000 películas de la IMDB. Probamos el comando que nos recomienda para ver las películas: 

```bash
curl 'https://ygvtpucoxveebizknhat.supabase.co/rest/v1/movies?select=*' \
-H "apikey: SUPABASE_KEY" \
-H "Authorization: Bearer SUPABASE_KEY"
```

Lo podemos probar también en Postman. Esta información proporcionada por el `API Docs` de Supabase será la base para una función inicial de obtención de las películas:

```javascript
const getSupabase = async (table) => {
  try {
    let response = await fetch(
      `https://ygvtpucoxveebizknhat.supabase.co/rest/v1/${table}?select=*`,
      {
        headers: {
          apikey,
          Authorization: `Bearer ${apikey}`,
        },
      },
    );
    if (!response.ok) {
      return Promise.reject("Bad request");
    }
    return response;
  } catch {
    return Promise.reject("Network Error");
  }
};

const getData = (response) => {
  return response.json();
};

```

Como esta función, podemos ir haciendo las demás. 

Veamos un objetoi de los que retorna esta petición:

```json
{
		"adult": false,
		"belongs_to_collection": "Toy Story Collection",
		"budget": "30000000",
		"original_language": "en",
		"original_title": "Toy Story",
		"overview": "Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.",
		"popularity": 21.946943,
		"release_date": "1995-10-30",
		"revenue": "373554033.0",
		"runtime": "81.0",
		"tagline": "not available",
		"title": "Toy Story",
		"vote_average": "7.7",
		"vote_count": "5415.0",
		"languages": "['English']",
		"day_of_week": "Monday",
		"month": "Oct",
		"season": "Q4",
		"year": "1995",
		"has_homepage": "YES",
		"genre": "['Animation', 'Comedy', 'Family']",
		"companies": "['Pixar Animation Studios']",
		"countries": "['United States of America']",
		"id": "8554eb64-1588-4480-84aa-ab1796b1707b"
}
```

Como se puede ver, es un objeto que tiene un "id" generado automáticamente por Supabase que es lo que lo identifica en la base de datos. Uno de los problemas que tiene es que el género, compañías, idiomas y países son "arrays", pero tratados por Supabase como Strings. Por tanto, si queremos interpretarlos hay que parsearlos. Por lo demás no parece tener más problemas. 

Luego, para ver las películas de forma básica en la vista, gracias a `Bootstrap` podemos hacer una lista desplegable: 


